In [1]:
import json
import pandas as pd
import numpy
from pathlib import Path
from rapidfuzz import fuzz
from docutils.nodes import label

Chuẩn bị data đã được kiểm tra brand 

JSON

In [3]:
path_excel = r"C:\Users\dongv\Downloads\test_json\sữa bầu.xlsx"
df = pd.read_excel(path_excel)
df.head(5)

,product_name,url_thumbnail,brand clean
0,[COMBO CHO MẸ BẦU] Sữa Hạt Nguyên Chất 137 Deg...,https://vn-live-01.slatic.net/p/c90fafac1890ac...,137 degrees
1,"Sữa Hạt 24 Grains Vị Cacao Hộp 700G, Sữa Thực ...",https://vn-live-01.slatic.net/p/2f55cd2429d957...,24 grains
2,"Sữa gạo hữu cơ vani 4Care Balance 180ml, sữa t...",https://vn-live-01.slatic.net/p/c3ab3ff9ca6949...,4care balance
3,Sữa Hạt Vị Tảo CHÍNH HÃNG - A+ FOOD Hộp 15 gói...,https://cf.shopee.vn/file/vn-11134207-7qukw-lg...,a+food
4,Sữa bầu A2 Pregnancy Platinum Premiuim úc 900g,https://cf.shopee.vn/file/e305ce279512b95ddb9a...,a2


In [8]:
dict_value_commodity_group = {
    "sữa" : "1__00",
    "điện máy": "2__00",
    "nước": "3__00",
}

In [19]:
def write_json_file(commodity_group, product_industry, path_excel, data_dict):
    df = pd.read_excel(path_excel)
    value_commodity_group = None
    for key, val in dict_value_commodity_group.items():
        if commodity_group == key:
            value_commodity_group = f"{val}"
     
    commodity_group_found = False
    commodity_group_item = None
    for item in data_dict:
        if item['label'] == commodity_group:
            commodity_group_found = True
            commodity_group_item = item
            break
    if not commodity_group_found:
        commodity_group_item = {
            'value': f"{value_commodity_group}{len(data_dict)+1}",
            'label': commodity_group,
            'product_industry': []
        }
        data_dict.append(commodity_group_item)

    product_industry_exists = False
    for industry in commodity_group_item['product_industry']:
        if industry['label'] == product_industry:
            product_industry_exists = True
            break
    
    if not product_industry_exists:
        commodity_group_item['product_industry'].append({
            'value': f"{commodity_group_item['value']}{len(commodity_group_item['product_industry'])+1}",
            'label': product_industry,
            'list_brand': []
        })

    for idx, row in df.iterrows():
        brand_cleand = str(row["brand clean"]).lower()
        image = str(row["url_thumbnail"])
        
        
        for industry in commodity_group_item['product_industry']:
            if industry['label'] == product_industry:
                if brand_cleand not in [brand['brand'] for brand in industry['list_brand']]:
                    industry['list_brand'].append({
                        'value' : f"{industry['value']}{len(industry["list_brand"])+1}",
                        'brand': brand_cleand,
                        'image': image
                    })

    return data_dict
data_dict = []
path_excel = r"C:\Users\dongv\Downloads\test_json\sữa bầu.xlsx"
data_dict = write_json_file("sữa", "sữa bầu",path_excel, data_dict)
data_dict


[{'value': '1__001',
  'label': 'sữa',
  'product_industry': [{'value': '1__0011',
    'label': 'sữa bầu',
    'list_brand': [{'value': '1__00111',
      'brand': '137 degrees',
      'image': 'https://vn-live-01.slatic.net/p/c90fafac1890ac0f24a63a5e639c87a0.png'},
     {'value': '1__00112',
      'brand': '24 grains',
      'image': 'https://vn-live-01.slatic.net/p/2f55cd2429d9577acf5426373c0f6b73.jpg'},
     {'value': '1__00113',
      'brand': '4care balance',
      'image': 'https://vn-live-01.slatic.net/p/c3ab3ff9ca6949bfad80e43e4055d804.png'},
     {'value': '1__00114',
      'brand': 'a+food',
      'image': 'https://cf.shopee.vn/file/vn-11134207-7qukw-lgtf61wfztv7d8'},
     {'value': '1__00115',
      'brand': 'a2',
      'image': 'https://cf.shopee.vn/file/e305ce279512b95ddb9a15f3a9831894'},
     {'value': '1__00116',
      'brand': 'alisa',
      'image': 'https://cf.shopee.vn/file/1cb7b64113b7db75599295a774e9e5e5'},
     {'value': '1__00117',
      'brand': 'alpha lipid',
  

ĐỌC JSON

In [17]:
output_json_path = r"C:\Users\dongv\Downloads\test_json\brand.json"
with open(output_json_path, 'w', encoding='utf-8') as json_file:
    json.dump(data_dict, json_file, ensure_ascii=False, indent=4)

ADD JSON

In [24]:
path_json = r"C:\Users\dongv\Downloads\test_json\brand.json"
path_excel_1 = r"C:\Users\dongv\Downloads\test_json\sữa công thức.xlsx"

with open(path_json, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)
data_dict = write_json_file("sữa","sữa công thức" , path_excel_1, data_dict)
with open(path_json, 'w', encoding='utf-8') as json_file:
    json.dump(data_dict, json_file, ensure_ascii=False, indent=4)
    

Lấy brand trong data

In [110]:
path_json = r"C:\Users\dongv\Downloads\test_json\brand.json"
with open(path_json, 'r', encoding='utf-8') as json_file:
    data_check = json.load(json_file)
    
list_row = []
dict_row = {}
for industry_data in data_check:
    product_industry_list = industry_data.get('product_industry', [])
    for industry in product_industry_list:
        for lst_brand in industry['list_brand']:
            dict_row = {
                'value' : lst_brand['value'],
                'brand' : lst_brand['brand'],
                'image' : lst_brand['image']
            }
            list_row.append(dict_row)


[{'value': '1__00111',
  'brand': '137 degrees',
  'image': 'https://vn-live-01.slatic.net/p/c90fafac1890ac0f24a63a5e639c87a0.png'},
 {'value': '1__00112',
  'brand': '24 grains',
  'image': 'https://vn-live-01.slatic.net/p/2f55cd2429d9577acf5426373c0f6b73.jpg'},
 {'value': '1__00113',
  'brand': '4care balance',
  'image': 'https://vn-live-01.slatic.net/p/c3ab3ff9ca6949bfad80e43e4055d804.png'},
 {'value': '1__00114',
  'brand': 'a+food',
  'image': 'https://cf.shopee.vn/file/vn-11134207-7qukw-lgtf61wfztv7d8'},
 {'value': '1__00115',
  'brand': 'a2',
  'image': 'https://cf.shopee.vn/file/e305ce279512b95ddb9a15f3a9831894'},
 {'value': '1__00116',
  'brand': 'alisa',
  'image': 'https://cf.shopee.vn/file/1cb7b64113b7db75599295a774e9e5e5'},
 {'value': '1__00117',
  'brand': 'alpha lipid',
  'image': 'https://cf.shopee.vn/file/vn-11134201-7r98o-lnclxz195zzu4c'},
 {'value': '1__00118',
  'brand': 'amanda',
  'image': 'https://filebroker-cdn.lazada.vn/kf/S311ddc80733e4908801903c3438b1781r.jp